In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import os
from torchinfo import summary

In [22]:
SAVE_MODEL = r"C:\Users\KDP15\Desktop\딥러닝프로젝트\은닉층(3개)피쳐1개제외,학습률0.0005\모델\best_model_epoch_28"

In [5]:
class MCFmodel(nn.Module):
    def __init__(self, in_in, out_out, h_list=[]):
        super().__init__()

        self.input_layer = nn.Linear(in_in,h_list[0])      #입력은 피쳐수
        self.h_layers = nn.ModuleList()

        for i in range(len(h_list)-1):
            self.h_layers.append(nn.Linear(h_list[i],h_list[i+1]))
        
        self.out_layers = nn.Linear(h_list[-1], out_out)  # 타겟수
    
    def forward(self, x):
        y = F.relu(self.input_layer(x))  

        for h_layer in self.h_layers:
            y=F.relu(h_layer(y))   
        
        return self.out_layers(y) # 다중분류

In [23]:
# 커스텀모델은 모델을 임포트해야 가능하다
if os.path.exists(SAVE_MODEL): # 해당경로에 있다면
    model= torch.load(SAVE_MODEL, weights_only=False)
else:
    print(f'{SAVE_MODEL} 파일이 존재하지 않습니다. 다시 확인하세요. ') 

In [24]:
summary(model)

Layer (type:depth-idx)                   Param #
MCFmodel                                 --
├─Linear: 1-1                            224
├─ModuleList: 1-2                        --
│    └─Linear: 2-1                       528
│    └─Linear: 2-2                       136
│    └─Linear: 2-3                       36
├─Linear: 1-3                            15
Total params: 939
Trainable params: 939
Non-trainable params: 0

In [33]:
data = [float(x) for x in input("player_assists,player_dbno, player_dist_walk, player_dmg, player_kills, player_survive_time ").split(',')]

In [34]:
dataTS = torch.FloatTensor(data).reshape(1,-1)
dataTS.shape, dataTS

(torch.Size([1, 6]),
 tensor([[0.0000e+00, 1.0000e+00, 2.5000e-01, 1.6500e+02, 0.0000e+00, 3.4000e+02]]))

In [35]:
# 새로운 데이터에 대한 예측 즉, predict
model.eval()
with torch.no_grad():

    # 추론/평가
    pre_val=model(dataTS)

In [36]:
pre_val

tensor([[ 4489.7349, -5962.9331, -7212.8169]])

In [37]:
class_names = ['상위10프로', '상위50프로', '분발해주세요']

In [38]:
torch.argmax(pre_val, dim=1).item()

0

In [39]:
predict = class_names[torch.argmax(pre_val, dim=1).item()]

In [40]:
print(f'당신의 예상 순위는 {predict}% 안에 있을겁니다')  

당신의 예상 순위는 상위10프로% 안에 있을겁니다
